In [1]:
import pandas as pd
import numpy as np
import string
import re
import json

In [86]:
tweet_dir = '/scratch/dzhang5/usda_project/tweet-fid-application'
output_text_prompt_path = '/scratch/dzhang5/LLM/llama/llama/food_prompts.json'
output_chat_prompt_path = '/scratch/dzhang5/LLM/llama/llama/food_chat_prompts_4.json'

In [3]:
train_mv_data = pd.read_pickle(f"{tweet_dir}/tweet-fid/LREC_mv/train.p")
train_expert_data = pd.read_pickle(f"{tweet_dir}/tweet-fid/LREC_expert_label/train.p")
train_bsc_data = pd.read_pickle(f"{tweet_dir}/tweet-fid/LREC_BSC/train.p")
unlabeled_data = pd.read_pickle(f"{tweet_dir}/tweet-fid-unlabeled.p")

In [4]:
train_bsc_data.columns

Index(['id', 'tweet', 'tweet_tokens', 'relevant_entity_label', 'entity_label',
       'sentence_class'],
      dtype='object')

In [5]:
clean_train_data = train_mv_data.loc[(train_mv_data['relevant_entity_label'] == train_expert_data['relevant_entity_label'])&(train_mv_data['sentence_class']==train_expert_data['sentence_class'])]

relevant entity in relevant sentence 4\
irrelevant entity in relevant sentence 1\
irrelevant entity in irrelevant sentence 1\
no entity in relevant sentence 1 \
no entity in irrelevant sentence 1

In [31]:
def assign_single_label(token_list, label_list, label_type):
    output_list = []
    for i, (token, label) in enumerate(zip(token_list, label_list)):
        if label == f'B-{label_type}':
            new_token = f'$${token}'
            if (i == len(label_list) - 1) or (label_list[i+1] != f'I-{label_type}'):
                new_token = new_token + "##"
            output_list.append(new_token)
        elif label == f'I-{label_type}':
            new_token = token
            if (i == len(label_list) - 1) or (label_list[i+1] != f'I-{label_type}'):
                new_token = new_token + "##"
            output_list.append(new_token)
        else:
            output_list.append(token)
    return ' '.join(output_list)

In [7]:
def find_conflict(entity_list, relevant_list, label_type):
    for entity, relevant in zip(entity_list, relevant_list):
        if entity.endswith(label_type) and relevant == 'O':
            return True
    return False

In [8]:
def find_special_words(text):
    # Regular expression pattern: $$ followed by any characters and ending with ##
    pattern = r'\$\$.*?##'
    return re.findall(pattern, text)

In [9]:
def find_punctuation(word_list):
    punctuation_marks = set(string.punctuation)
    found_punctuation = [word for word in word_list if word in punctuation_marks]
    return found_punctuation

# Example usage
word_list = ['When', 'you', 'doing', 'it', 'in', 'the', 'dark', 'and', 'he', 'slap', 'yo', 'stomach', 'and', 'says', '"', 'you', 'have', 'a', 'fat', 'ass', '"', '.']
punctuation = find_punctuation(word_list)
print(punctuation)


['"', '"', '.']


In [10]:
unlabeled_data['tweet_text'].apply(find_special_words).explode().unique()

array([nan], dtype=object)

In [11]:
train_mv_data['tweet'].apply(find_special_words).explode().unique()

array([nan], dtype=object)

In [12]:
unlabeled_data['tweet_token'].apply(find_punctuation).explode().unique()

array([nan, '?', '-', '.', '=', '/', ',', '!', '~', '@', '+', '#', '"',
       '|', '(', ':', '*', '$', '^', ';', ')', "'", '`', '[', ']', '%',
       '{', '}'], dtype=object)

In [13]:
unlabeled_data['token_prediction'].apply(lambda x: x.split(',')).explode().unique()

array(['O', 'I-loc', 'I-food', 'I-symptom', 'B-symptom', 'B-food',
       'B-loc', 'B-other', 'I-other'], dtype=object)

In [14]:
keep_data = train_expert_data.loc[train_expert_data.apply(lambda x: find_conflict(x['entity_label'], x['relevant_entity_label'], 'food'), axis=1)&train_expert_data['sentence_class']==1]['tweet']
for tweet, i in zip(keep_data, keep_data.index):
    print(i, tweet)

1049454650661855232 The first time i had cheese burger 6 years ago,the next day tu terus food poisoning because i cant eat this pickle. Lepas dari tu mmg tak makan dah cheese burger mcd. Huhu
1203040895106461697 Other - Harrisburg, Illinois - I ate full meal at community "soup kitchen" yesterday - nothing else until early morning when I ate leftover green b... Food Poisoning  HTTPURL  HTTPURL
1263550719984046080 tw food related//  my dad just asked me if im feeling nauseous bc apparently him and mom have been feeling similar :D so like. im assuming there was something wrong w the fish and Hoping. none of us gets food poisoning this really isn't the time to go to an hospital
1049010230816857856 @USER got food poisoning from fish many years back and nearly died - i mean i'll still eat it - haddock, scampi, stuff like that, but not overly keen in general
1060286815599583237 Tmi / last night I experienced the quickest and most brutal food poisoning I've ever experienced and I've had bad fo

relevant food: 1047692408203296768 1055501133052104705 1137088028059377664

relevant location: 1309250211831832576  983799838679715842

relevant symptom: 1159177181672742918 1313592751225794560

relevant other: 1060664485466329088 1061331199300386817

irrelevant food in relevant tweet: 851534611876282368 1025851135523991553

irrelevant location in relevant tweet: 1337821380340015107 1340309125515886593

irrelevant symptom in relevant tweet: 

irrelevant other in relevant tweet: 

irrelevant food in irrelevant tweet: 973942771663998977 1025843082720239616

irrelevant location in irrelevant tweet: 482134863332397056 958472652414369793

irrelevant symptom in irrelevant tweet: 1421084359189221377 1414751071914893313

irrelevant other in irrelevant tweet: 1061174394775584768 547853139815460864

irrelevant O in irrelevant tweet: 1383208546402193413 1412041021668552705

In [15]:
# relevant entity in relevant sentence 
keep_data = clean_train_data.loc[clean_train_data['relevant_entity_label'].apply(lambda x: any([i=='B-food' for i in x]))]['tweet']
for tweet, i in zip(keep_data, keep_data.index):
    print(i, tweet)

960617307675521024 I'm never eating shit I didn't make again, this fast food giving me food poisoning
1060664485466329088 i'm pretty sure i got food poisoning from the cafeteria at work. in my defense you can't SMELL the lettuce/mushrooms before you buy them there's a SNEEZE GUARD you can't tell if they've gone off
1263577222616662019 @USER Haha, sorry. It didn't smell great, but I knew it wasn't great as soon as I cut into it and water started gushing out. That must have some nasty watermelon to get food poisoning from.
1060285550685384704 The service was bad, the food poisoning was worse, and your online review said so. Now you're facing something even tougher to stomach: a lawsuit from the restaurant you posted about. #lifeisunpredictable #TimberlineIns  Timberline Insurance Agency/ Che... HTTPURL HTTPURL
286180260356169728 Biggest regret of #2013: eating sushi from safeway's refrigerated section #foodpoisoning #projectilevomitingatmidnight
40874543140700160 At least Im not knocked 

In [16]:
# irrelevant entity in relevant sentence
keep_data = clean_train_data.loc[clean_train_data.apply(lambda x: find_conflict(x['entity_label'], x['relevant_entity_label'], 'food'), axis=1)&clean_train_data['sentence_class']==1]['tweet']
for tweet, i in zip(keep_data, keep_data.index):
    print(i, tweet)

1309269481596878848 any other vegans or people in general get really sick after they eat beyond “meat”?? i've tried it twice and every time i get awful food poisoning.
1114312893070286848 Today's sandwich menu cordially offers sautéed salmon with a side of scrumptious food poisoning!
1054903141215490048 When there's a possibility that the cafeteria gave you food poisoning and you only have $2 until payday, you eat the Ramen noodles. Might not fill you up, but at least it's something. #brokecollegekid
1279129708773466114 I got food poisoning and i just wanna say I'm never eating canes again
851534611876282368 Day 1 of my no carb diet and I get food poisoning from undercooked meat at a cookout. Is this a sign? A donut has never done this to me.
1060656422709538816 @USER one of my homegirls was throwing up/nauseous her whole trip i felt sooooo bad. but at the end of the day you are effectively food poisoning yourself & everyone's body cant take that. maybe try microdosing it into ginger t

In [17]:
# irrelevant entity in irrelevant sentence
keep_data = clean_train_data.loc[clean_train_data['entity_label'].apply(lambda x: any([i=='B-symptom' for i in x]))&(clean_train_data['sentence_class']==0)]['tweet']
for tweet, i in zip(keep_data, keep_data.index):
    print(i, tweet)

1159033913844494338 More shit my boss says   Me post moving: If I fall ill from food poisoning I can have a 5 day weekend  My boss without skipping a beat: Don't die on me!
1421084359189221377 officially can't open my windows bc of the dust clouds from the fire... feeling sick to my stomach
974774050697498624 @USER Put it this way: If we stick with the UK rather than EU, we'll be closer aligned with the US food and drink standards which means accepting part of an industry that has near 12x the annual incidences of food poisoning. Do you like vomit and diarrhoea?   HTTPURL
1414753516816642048 i overate fucking fr0sted wheat cereal i do this every fucking time my stomach hurts so bad rn
1350769285942169601 Hello all,it's still me..re my new profile pic,3 days..3 days to inauguration day.I feel sick to my stomach God only knows how the good ppl of America feel..I could never watch that satanic bael worshipping shitshow for all the money in China...excuse the pun
1396478235383304194 If I a

In [18]:
# no entity in irrelevant sentence
keep_data = clean_train_data.loc[clean_train_data['entity_label'].apply(lambda x: all([i=='O' for i in x]))&(clean_train_data['sentence_class']==0)]['tweet']
for tweet, i in zip(keep_data, keep_data.index):
    print(i, tweet)

1396479336815992834 I got to stop eating in the middle of the night my stomach be fucked up!!!
1362175195847536644 @USER Hello. Im asking for help with food please, we don't  have anything  to eat at home. Mama can't  even take her pills on an empty stomach please it hurts  to see the younger ones go hungry. They say their tummy hurts please any help will be appreciated . God bless u 081 7337 617
1025696364003512320 #marijuana #cannabis #weed #food #poison #wakeup #laws #flaws #naturalhealer @ Cortland, New York  HTTPURL
1337821380340015107 @USER . . . back a little ways, then lounge himself forward off his toes at his counterpart to give that stomach some more punishment .  A flurry of uppercuts to it before ending with a strong hook to his visor .  ❛❛ I'm not gonna let you torment me anymore ! ❜❜   (+)
1383208546402193413 Wait way it goes I might turn off the video playing on here I can't stomach much more of these videos
1313584523645464578 @USER @USER 28 WEEKS for an abortion? YEAH

In [19]:
# no entity in relevant sentence
keep_data = clean_train_data.loc[clean_train_data['entity_label'].apply(lambda x: all([i!='B-food' for i in x]))&(clean_train_data['sentence_class']==1)]['tweet']
for tweet, i in zip(keep_data, keep_data.index):
    print(i, tweet)

855883668933541889 @USER Food poisoning!!!! Please I've actually experiencing it right when I tweet this
1078743706847436800 The first episode of @USER  I ever watched was on Thanksgiving 2018 when I had severe food poisoning. I have now watched it 3 times through. It's been a month. At this rate they should just ask me to be on the show.
785983115072339972 @USER you just disappeared after you and @USER gave me food poisoning smh
1073699596319129600 If she also had suffered diarrhea, immediate hospitalization can't always help. I almost died from food poisoning in only 17 hours total, because of dehydration even with 6 of those hours in the hospital! The parents caused this, not the USA!
855854718890987520 Can't believe I've got food poisoning
835182661543030785 One of y'all mfs gave me food poisoning. Since I can't narrow it down I'll just blame everybody lol
261246875712249857 @USER I want it fresh! Tht was a week ago. Thts how you get food poison lol
1075486487867731969 Other - Fair

In [20]:
tweet_list = [1047692408203296768, 1055501133052104705, 1309250211831832576,  983799838679715842, 
              1159177181672742918, 1313592751225794560, 1060664485466329088, 1061331199300386817,
              973942771663998977, 1025843082720239616, 482134863332397056, 958472652414369793,
              1421084359189221377, 1414751071914893313, 1061174394775584768, 547853139815460864,
              1383208546402193413, 1412041021668552705, 851534611876282368, 1025851135523991553]

In [21]:
for tweet, i in zip(clean_train_data.loc[tweet_list]['tweet'], tweet_list):
    print(i, tweet)

1047692408203296768 i ate what i thought was delicious chicken from soonta and now i feel ridiculously nauseous, lightheaded and just plain ill. i've never had food poisoning so can someone diagnose me
1055501133052104705 I ate a sausage mcmuffin from your store at 2422 W Pioneer Pkwy, Pantego, TX 76013 and got food poisoning so I missed 2 days of work... @USER
1309250211831832576 Run away from LilyGate Hotel guyss. My sister went there on Sunday with her friends and all 8 of them got food poisoning, one person got admitted to the hospital today and when they complained, they were offered food vouchers instead. Run ohh
983799838679715842 Got Pizza hut for dinner! I ate 4 normal slices. Then im throwing up! Bye Pizza
1159177181672742918 Taco Bell - Phoenix, Arizona - I had a beer supreme chalupa and a soft taco supreme around 7pm on August 6, 2019 and started feeling nauseous around 8:30pm.... Food Poisoning  HTTPURL  HTTPURL
1313592751225794560 I took a bite of a curry chicken wrap and

In [22]:
for tweet, i in zip(clean_train_data.loc[tweet_list]['entity_label'], tweet_list):
    print(i, tweet)

1047692408203296768 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-food', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'O', 'B-symptom', 'O', 'B-symptom', 'O', 'O', 'O', 'B-symptom', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O']
1055501133052104705 ['O', 'O', 'O', 'B-food', 'I-food', 'O', 'O', 'O', 'O', 'B-loc', 'I-loc', 'I-loc', 'I-loc', 'O', 'B-loc', 'O', 'B-loc', 'I-loc', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
1309250211831832576 ['O', 'O', 'O', 'B-loc', 'I-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
983799838679715842 ['O', 'B-loc', 'I-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-symptom', 'I-symptom', 'O', 'O', 'B-food']
1159177181672742918 ['B-loc', 'I-loc', 'O', 'B-loc', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'B-food', 'I-food', 'I

In [23]:
for tweet, i in zip(clean_train_data.loc[tweet_list]['relevant_entity_label'], tweet_list):
    print(i, tweet)

1047692408203296768 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-food', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'O', 'B-symptom', 'O', 'B-symptom', 'O', 'O', 'O', 'B-symptom', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O']
1055501133052104705 ['O', 'O', 'O', 'B-food', 'I-food', 'O', 'O', 'O', 'O', 'B-loc', 'I-loc', 'I-loc', 'I-loc', 'O', 'B-loc', 'O', 'B-loc', 'I-loc', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
1309250211831832576 ['O', 'O', 'O', 'B-loc', 'I-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-other', 'I-other', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
983799838679715842 ['O', 'B-loc', 'I-loc', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-symptom', 'I-symptom', 'O', 'O', 'B-food']
1159177181672742918 ['B-loc', 'I-loc', 'O', 'B-loc', 'O', 'B-loc', 'O', 'O', 'O', 'O', 'B-food', 'I-food', 'I-foo

In [24]:
for tweet, i in zip(clean_train_data.loc[tweet_list]['tweet_tokens'], tweet_list):
    print(i, tweet)

1047692408203296768 ['i', 'ate', 'what', 'i', 'thought', 'was', 'delicious', 'chicken', 'from', 'soonta', 'and', 'now', 'i', 'feel', 'ridiculously', 'nauseous', ',', 'lightheaded', 'and', 'just', 'plain', 'ill', '.', "i've", 'never', 'had', 'food', 'poisoning', 'so', 'can', 'someone', 'diagnose', 'me']
1055501133052104705 ['I', 'ate', 'a', 'sausage', 'mcmuffin', 'from', 'your', 'store', 'at', '2422', 'W', 'Pioneer', 'Pkwy', ',', 'Pantego', ',', 'TX', '76013', 'and', 'got', 'food', 'poisoning', 'so', 'I', 'missed', '2', 'days', 'of', 'work', '.', '.', '.', '@USER']
1309250211831832576 ['Run', 'away', 'from', 'LilyGate', 'Hotel', 'guyss', '.', 'My', 'sister', 'went', 'there', 'on', 'Sunday', 'with', 'her', 'friends', 'and', 'all', '8', 'of', 'them', 'got', 'food', 'poisoning', ',', 'one', 'person', 'got', 'admitted', 'to', 'the', 'hospital', 'today', 'and', 'when', 'they', 'complained', ',', 'they', 'were', 'offered', 'food', 'vouchers', 'instead', '.', 'Run', 'ohh']
983799838679715842 [

In [25]:
clean_train_data[['sentence_class']]

,sentence_class
id,
1047682026822762496,0
1348197325576003587,0
1061174394775584768,0
1396479336815992834,0
960617307675521024,1
...,...
1356835307044278273,0
656859029822664704,0
256547773808250880,0


In [26]:
unlabeled_data.columns

Index(['id', 'author_id', 'tweet_text', 'created_at', 'geo', 'user',
       'tweet_token', 'token_prediction', 'sentence_prediction',
       'sentence_prediction_prob', 'symptoms', 'locations', 'foods', 'others',
       'tweet_len', 'entities_cnt'],
      dtype='object')

In [57]:
output_pos_list = [1047692408203296768, 1055501133052104705, 1159177181672742918, 1137088028059377664] 
output_neg_list = [1025851135523991553, 973942771663998977, 1421084359189221377, 1091809221891506176]
output_pos_list = [1047692408203296768, 1137088028059377664] 
output_neg_list = [1025851135523991553, 973942771663998977]
# output_pos_list = [1047692408203296768] 
# output_neg_list = [1025851135523991553]
output_list = [item for pair in zip(output_pos_list, output_neg_list) for item in pair]

In [58]:
pos_entity_unlabeled = unlabeled_data.loc[unlabeled_data['token_prediction'].str.contains('B-food')]['tweet_token'].apply(lambda x: " ".join(x))
neg_entity_unlabeled = unlabeled_data.loc[~unlabeled_data['token_prediction'].str.contains('B-food')]['tweet_token'].apply(lambda x: " ".join(x))

In [59]:
label_sentence = pd.concat([pos_entity_unlabeled.sample(n=5,random_state=10), neg_entity_unlabeled.sample(n=5,random_state=10)])

In [56]:
entity_type = 'food'
sub_prompt = 'cause a potential foodborne illness case in the given tweet'
prompt = f"I am an excellent linguist, The task is to label {entity_type} entities that {sub_prompt}. Below are some examples."
input_sentence_list = clean_train_data.loc[output_list]['tweet_tokens'].apply(lambda x: " ".join(x))
output_sentence_list = clean_train_data.loc[output_list][['tweet_tokens', 'relevant_entity_label']].apply(lambda x: assign_single_label(x['tweet_tokens'], x['relevant_entity_label'], entity_type), axis=1)
for isen, osen in zip(input_sentence_list, output_sentence_list):
    prompt += f'\nInput: {isen} \nOutput: {osen}'
print(prompt)

output_sen_list = []
for sen in label_sentence:
    output_sen = prompt + f"\nInput: {sen} \nOutput:"""
    # print(output_sen)
    output_sen_list.append(output_sen)

In [69]:
with open(f"{output_text_prompt_path}", 'w') as output_file:
    json.dump(output_sen_list, output_file)

In [93]:
entity_type = 'food'
sub_prompt = 'cause a potential foodborne illness case in the given sentence'
prompt = f"You are an excellent linguist. Within the TWEET-FID dataset, the task is to label {entity_type} entities that {sub_prompt}. Below are some examples, and you should make the same prediction as the examples."
input_sentence_list = clean_train_data.loc[output_list]['tweet_tokens'].apply(lambda x: " ".join(x))
output_sentence_list = clean_train_data.loc[output_list][['tweet_tokens', 'relevant_entity_label']].apply(lambda x: assign_single_label(x['tweet_tokens'], x['relevant_entity_label'], entity_type), axis=1)
for isen, osen in zip(input_sentence_list, output_sentence_list):
    prompt += f'\nThe given sentence: {isen} \nThe labeled sentence: {osen}'
print(prompt)

output_dia_list = []
for sen in label_sentence:
    # output_sen = prompt + f"\nPlease make prediction for this sentence: {sen}"""
    output_sen = prompt + f"\nThe given sentence: {sen} \nThe labeled sentence:"
    # print(output_sen)
    # output_dia = [{"role":"system", "content":"Always only make prediction for the last sentence"}, {"role":"user", "content":output_sen}]
    output_dia = [{"role":"system", "content":"Always only return the requested labeled sentence for the user's last given sentence, don't say any other words like 'understood'."}, {"role":"user", "content":output_sen}]
    # output_dia = [{"role":"user", "content":output_sen}]
    output_dia_list.append(output_dia)

You are an excellent linguist. Within the TWEET-FID dataset, the task is to label food entities that cause a potential foodborne illness case in the given sentence. Below are some examples, and you should make the same prediction as the examples.
The given sentence: i ate what i thought was delicious chicken from soonta and now i feel ridiculously nauseous , lightheaded and just plain ill . i've never had food poisoning so can someone diagnose me 
The labeled sentence: i ate what i thought was delicious $$chicken## from soonta and now i feel ridiculously nauseous , lightheaded and just plain ill . i've never had food poisoning so can someone diagnose me
The given sentence: They say after having food poisoning you should take it easy and slowly work your way back into solid regular food . . . . yeahhh oookkk , I told my stomach you better suck it up and get ready for some real Dominican dishes anit nobody got time to baby you lol 
The labeled sentence: They say after having food poisoni

In [94]:
with open(f"{output_chat_prompt_path}", 'w') as output_file:
    json.dump(output_dia_list, output_file)

In [9]:
entity_type = 'location'
sub_prompt = 'cause a potential foodborne illness case in the given sentence'
prompt = f"You are an excellent linguist. Within the TWEET-FID dataset, the task is to label {entity_type} entities. Below are some examples.\n"
text = "Input: Azerbaijan beat Switzerland 1-0 ( halftime 1-0 ) in their World Cup soccer European group three qualifying match on Saturday .\n\
Output: @@Azerbaijan## beat @@Switzerland## 1-0 ( halftime 1-0 ) in their World Cup soccer European group three qualifying match on Saturday .\n\
Input: Nijmeh of Lebanon beat Nasr of Saudi Arabia 1-0 ( halftime 1-0 ) in their Asian club championship second round first leg tie on Saturday .\n\
Output: Nijmeh of @@Lebanon## beat Nasr of @@Saudi Arabia## 1-0 ( halftime 1-0 ) in their Asian club championship second round first leg tie on Saturday .\n\
Input: Slovakia beat the Faroe Islands 2-1 ( halftime 1-0 ) in their World Cup soccer European group six qualifying match on Saturday .\n\
Output: @@Slovakia## beat the @@Faroe Islands## 2-1 ( halftime 1-0 ) in their World Cup soccer European group six qualifying match on Saturday .\n\
Input: Canada beat Panama 3-1 ( halftime 2-0 ) in their CONCACAF semifinal phase qualifying match for the 1998 World Cup on Friday .\n\
Output: @@Canada## beat @@Panama## 3-1 ( halftime 2-0 ) in their CONCACAF semifinal phase qualifying match for the 1998 World Cup on Friday .\n\
Input: Soccer - Azerbaijan beat Switzerland in world cup Qualifier .\n\
Output: Soccer - @@Azerbaijan## beat @@Switzerland## in world cup Qualifier .\n\
Input: Soccer - Wales beat San Marino in world cup Qualifier .\n\
Output: Soccer - @@Wales## beat @@San Marino## in world cup Qualifier .\n\
Input: The United States edged Austria in Salzburg 3-2 in the opening round in April , and then blanked Japan 5-0 in Nagoya last month in the semifinals .\n\
Output: The @@United States## edged @@Austria## in @@Salzburg## 3-2 in the opening round in April , and then blanked @@Japan## 5-0 in @@Nagoya## last month in the semifinals .\n\
Input: Soccer - Slovakia beat Faroes in world cup Qualifier .\n\
Output: Soccer - @@Slovakia## beat @@Faroes## in world cup Qualifier .\n\
In my next message, I will give you a sentence, and you should make the same prediction as the examples."

prompt = prompt + text
print(prompt)

output_dia_list = []
output_sen = "Input: Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .\n\
Output:"

# print(output_sen)
# output_dia = [{"role":"system", "content":"Always only make prediction for the last sentence"}, {"role":"user", "content":output_sen}]
output_dia = [{"role":"system", "content":"Always only return the requested labeled sentence for the user's last given sentence, don't say any other words or explain the reason for label"},
              {"role":"user", "content":prompt},
              {"role":"assistant", "content":"Understood! I will only return the labeled sentence for the user's last given sentence."},
              {"role":"user", "content":output_sen}]
# output_dia = [{"role":"user", "content":output_sen}]
output_dia_list.append(output_dia)

You are an excellent linguist. Within the TWEET-FID dataset, the task is to label location entities. Below are some examples.
Input: Azerbaijan beat Switzerland 1-0 ( halftime 1-0 ) in their World Cup soccer European group three qualifying match on Saturday .
Output: @@Azerbaijan## beat @@Switzerland## 1-0 ( halftime 1-0 ) in their World Cup soccer European group three qualifying match on Saturday .
Input: Nijmeh of Lebanon beat Nasr of Saudi Arabia 1-0 ( halftime 1-0 ) in their Asian club championship second round first leg tie on Saturday .
Output: Nijmeh of @@Lebanon## beat Nasr of @@Saudi Arabia## 1-0 ( halftime 1-0 ) in their Asian club championship second round first leg tie on Saturday .
Input: Slovakia beat the Faroe Islands 2-1 ( halftime 1-0 ) in their World Cup soccer European group six qualifying match on Saturday .
Output: @@Slovakia## beat the @@Faroe Islands## 2-1 ( halftime 1-0 ) in their World Cup soccer European group six qualifying match on Saturday .
Input: Canada b

In [11]:
output_dia_list

[[{'role': 'system',
   'content': "Always only return the requested labeled sentence for the user's last given sentence, don't say any other words or explain the reason for label"},
  {'role': 'user',
   'content': 'You are an excellent linguist. Within the TWEET-FID dataset, the task is to label location entities. Below are some examples.\nInput: Azerbaijan beat Switzerland 1-0 ( halftime 1-0 ) in their World Cup soccer European group three qualifying match on Saturday .\nOutput: @@Azerbaijan## beat @@Switzerland## 1-0 ( halftime 1-0 ) in their World Cup soccer European group three qualifying match on Saturday .\nInput: Nijmeh of Lebanon beat Nasr of Saudi Arabia 1-0 ( halftime 1-0 ) in their Asian club championship second round first leg tie on Saturday .\nOutput: Nijmeh of @@Lebanon## beat Nasr of @@Saudi Arabia## 1-0 ( halftime 1-0 ) in their Asian club championship second round first leg tie on Saturday .\nInput: Slovakia beat the Faroe Islands 2-1 ( halftime 1-0 ) in their World

In [12]:
with open(f"{'/scratch/dzhang5/LLM/llama/llama/demonstration_cases.json'}", 'w') as output_file:
    json.dump(output_dia_list, output_file)